## Preparing the full dataset:

In [99]:
import pandas as pd
import os

base_path = r"C:\Users\aalrassi\Documents\anastasiawork\ML_and_DS\brazil_ecom\data\brazil_ecom"

files = {
    'customers': 'customers_dataset.csv',
    'geolocation': 'geolocation_dataset.csv',
    'order_items': 'order_items_dataset.csv',
    'order_payments': 'order_payments_dataset.csv',
    'order_reviews': 'order_reviews_dataset.csv',  
    'orders': 'orders_dataset.csv',
    'product_cat_name': 'product_category_name_translation.csv',
    'products': 'products_dataset.csv',
    'sellers': 'sellers_dataset.csv'
}

for var_name, file_name in files.items():
    file_path = os.path.join(base_path, file_name)
    globals()[var_name] = pd.read_csv(file_path)


In [100]:
for name in files.keys():
    df = globals()[name] 
    print(name, df.columns)

customers Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')
geolocation Index(['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng',
       'geolocation_city', 'geolocation_state'],
      dtype='object')
order_items Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')
order_payments Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')
order_reviews Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')
orders Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estim

In [101]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [102]:
orders['late_delivery'] = (
    orders['order_delivered_customer_date'] > orders['order_estimated_delivery_date']
).astype(int)


In [103]:
df = orders.merge(
    customers,
    on='customer_id',
    how='left'
)

In [104]:
order_items_agg = order_items.groupby('order_id').agg({
    'price': 'sum',            
    'freight_value': 'sum',    
    'product_id': 'count',    
}).rename(columns={'product_id': 'num_items'})

df = df.merge(order_items_agg, on='order_id', how='left')


In [105]:
order_payments_agg = order_payments.groupby('order_id').agg({
    'payment_value': 'sum',           # total paid
    'payment_installments': 'max',    # max installments
}).reset_index()

df = df.merge(order_payments_agg, on='order_id', how='left')


In [106]:
# For simplicity, take first seller per order (or you could compute distances per seller)
order_items_seller = order_items[['order_id', 'seller_id']].drop_duplicates(subset=['order_id'])
df = df.merge(order_items_seller, on='order_id', how='left')
df = df.merge(sellers, on='seller_id', how='left')

In [107]:
# Seller geolocation
seller_geo = geolocation.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().reset_index()
df = df.merge(seller_geo, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
df = df.rename(columns={'geolocation_lat': 'seller_lat', 'geolocation_lng': 'seller_lng'}).drop(columns=['geolocation_zip_code_prefix'])

# Customer geolocation
customer_geo = geolocation.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().reset_index()
df = df.merge(customer_geo, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
df = df.rename(columns={'geolocation_lat': 'customer_lat', 'geolocation_lng': 'customer_lng'}).drop(columns=['geolocation_zip_code_prefix'])


In [108]:
locs = df[['seller_lat','seller_lng','customer_lat','customer_lng']].copy()
locs.drop_duplicates(inplace=True)


In [109]:
order_items_products = order_items.merge(products, on='product_id', how='left')
order_items_products_agg = order_items_products.groupby('order_id').agg({
    'product_weight_g': 'mean',
    'product_length_cm': 'mean',
    'product_height_cm': 'mean',
    'product_width_cm': 'mean',
}).reset_index()

df = df.merge(order_items_products_agg, on='order_id', how='left')


In [110]:
products = products.merge(
    product_cat_name[['product_category_name', 'product_category_name_english']],
    on='product_category_name',
    how='left'
)


In [114]:
order_items_with_cat = order_items.merge(
    products[['product_id', 'product_category_name_english']],
    on='product_id',
    how='left'
)

order_cat = order_items_with_cat.groupby('order_id')['product_category_name_english'] \
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'unknown') \
    .reset_index()

df = df.merge(order_cat, on='order_id', how='left')


In [116]:
path = r'C:\Users\aalrassi\Documents\anastasiawork\ML_and_DS\brazil_ecom\data'
df.to_csv(f'{path}\\brazil_ecommerce.csv', index=False)